In [47]:
import spacy
from spacy import displacy

import tensorflow as tf
import keras

import pandas as pd
import numpy as np
import itertools

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as mpatches

import sklearn
from sklearn.preprocessing import CategoricalEncoder
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from keras.preprocessing.text import text_to_word_sequence

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, GRU, Activation, Dropout
from keras import layers
from keras.layers import Input, Concatenate
from keras.models import Model, load_model
from keras.optimizers import SGD, Adam
from keras.models import load_model
from keras import regularizers
from keras.utils import plot_model
import keras.backend as K
K.set_image_data_format('channels_last')
from hyperopt import fmin, hp, Trials, tpe, STATUS_OK
from sklearn.metrics import confusion_matrix


In [26]:
# Here are the functions to pre-process the data before feeding it

def split_data(data, training_size=0.8):
  """
  data: Pandas Dataframe
  training_size: proportion of the data to be used for training
  This function splits the data into training_set and test_set based on the given training_size
  Return: train_set and test_set as pandas DataFrame
  """
  return data[:int(training_size*len(data))], data[int(training_size*len(data)):]


In [38]:
# importer le csv, quelques erreurs pouvaient exister 

dataset_path = r'C:\Users\maxim\Desktop\Work\Hackaton\Dataset\20181204_mines-paristech_hackathon-input\hack_train.csv'
dataset_raw = pd.read_csv(dataset_path, header = 0,sep = ';', error_bad_lines = False,encoding = "ISO-8859-1")

# shuffles the dataset
dataset_raw = dataset_raw.sample(frac=1).reset_index(drop=True)
training_set, test_set = split_data(dataset_raw)

# fits the count vectorizer on training data
count_vec = CountVectorizer()
transformer = TfidfTransformer()

descriptions_training = training_set['description']
descriptions_training.fillna(value='', inplace=True)

count_vec.fit(descriptions_training)

transformer.fit(count_vec.transform(descriptions_training))



TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [39]:
# Using bag of words technique

def process_bag(descriptions):

    x = count_vec.transform(descriptions)
    x = transformer.transform(x)
    
    return x

In [40]:

def create_inputs_outputs(data):
    """
    data: pandas DataFrame, this could be either training_set or test_set
    This function will create input array X from the given dataset and will normalize 'Close' and 'Volume' between 0 and 1
    Return: 
    data.values -> the input except description
    description -> the preprocessed description
    and output  -> the 'sold' booleans
    """

    output = data['sold']

    descriptions = data['description']
    descriptions.fillna(value='', inplace=True)
    descriptions = process_bag(descriptions)
    
    # we have to drop not relevant input and put the description apart
    data = data.drop(['description','UniqueID','sold'],1)

    # here we transform the categorical inputs in one-hot vectors
    for name in (data.columns):
      if name not in ['startprice', 'description']:
        one_hot = pd.get_dummies(data[name], prefix = name)
        data = pd.concat([data, one_hot], axis=1)
        data = data.drop(name,1)

    return data.values, descriptions, output


In [41]:
# divides the dataset into training and test
training_set, test_set = split_data(dataset_raw)

#descriptions = training_set['description']
#descriptions.fillna(value='', inplace=True)
#descriptions = process_bag(descriptions)

#print(descriptions[2])

X_train, D_train, Y_train =  create_inputs_outputs(training_set)
X_test,D_test, Y_test = create_inputs_outputs(test_set)

features_shape = X_train.shape[1]
descriptions_shape = D_train.shape[1]

print(features_shape)
print(descriptions_shape)


29
918


In [56]:
def modele(l=100, m=50, n=30, o=20, dropout_rate = 0.2):
        input_features = Input((features_shape,))
        input_descriptions = Input((descriptions_shape,))
    
        desc = input_descriptions
        feat = input_features
        
        desc = Dense(int(l), activation='relu')(desc)
        desc = Dropout(dropout_rate)(desc)
        desc = Dense(int(m), activation='relu')(desc)
        
        x = Concatenate(axis = -1)([feat, desc])
        
        x = Dense(int(n), activation='relu')(x)
        x = Dropout(dropout_rate)(x)
        x = Dense(int(0), activation='relu')(x)
        x = Dense(1, activation='sigmoid')(x)

        # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
        model = Model(inputs = [input_features, input_descriptions], outputs = X, name='model')
        return model

In [57]:
#hyperparam_opt()

modele1 = modele(l=100,m=50,n=30,o=20, dropout_rate=0.2)
modele1.compile(optimizer = SGD(0.00001),loss = "binary_crossentropy", metrics = ["accuracy"])
history = modele1.fit(inputs = [X_train, D_train], y = Y_train, epochs = 100, batch_size = 1000, validation_data = ([X_test,D_test], Y_test))
modele1.evaluate(x = features_test, y = sold_test)

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.show()

AssertionError: 